# TODO: Title
**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of the TODO's and/or use more than one cell to complete all the tasks.

In [1]:
!pip install smdebug
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [3]:
from tqdm import tqdm
import sagemaker

from sagemaker.pytorch import PyTorch
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

role = sagemaker.get_execution_role()

## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `train_data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not acessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

In [3]:
import os
import json
import boto3

def download_and_arrange_data():
    s3_client = boto3.client('s3')

    with open('file_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items():
        print(f"Downloading Images with {k} objects")
        directory=os.path.join('train_data', k)
        if not os.path.exists(directory):
            os.makedirs(directory)
        for file_path in tqdm(v):
            file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
            s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name),
                             os.path.join(directory, file_name))

In [4]:
download_and_arrange_data()

100%|██████████| 1228/1228 [01:34<00:00, 12.95it/s]


100%|██████████| 2299/2299 [03:05<00:00, 12.39it/s]


100%|██████████| 2666/2666 [03:42<00:00, 11.97it/s]


100%|██████████| 2373/2373 [03:17<00:00, 11.99it/s]


100%|██████████| 1875/1875 [02:30<00:00, 12.47it/s]


## Dataset
**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).

In [47]:
import shutil
import random

os.makedirs("train")
os.makedirs("test")

for i in range(1,6):
    sf_path = os.path.join("train_data", str(i))
    train_sf = os.path.join("train", str(i))
    test_sf = os.path.join("test", str(i))
    os.makedirs(train_sf)
    os.makedirs(test_sf)

    images = []
    for image in os.listdir(sf_path):
        images.append(image)
        random.shuffle(images)
        train_images = images[:int(0.8 * len(images))]
        test_images = images[int(0.8 * len(images)):]

    for train_image in train_images:
        if train_image.endswith('.jpg'):
            src = os.path.join(sf_path, train_image)
            dst = os.path.join(train_sf, train_image)
            shutil.copy(src, dst)

    for test_image in test_images:
        if train_image.endswith('.jpg'):
            src = os.path.join(sf_path, test_image)
            dst = os.path.join(test_sf, test_image)
            shutil.copy(src, dst)

In [49]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix1 = "sagemaker/Final-Project/train"
prefix2 = "sagemaker/Final-Project/test"

train = sagemaker_session.upload_data(path="train", bucket=bucket, key_prefix=prefix1)
print("train: {}".format(train))

test = sagemaker_session.upload_data(path="test", bucket=bucket, key_prefix=prefix2)
print("test: {}".format(test))

train: s3://sagemaker-us-east-1-553171274417/sagemaker/Final-Project/train
test: s3://sagemaker-us-east-1-553171274417/sagemaker/Final-Project/test


## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [19]:
hyperparameters = {
            "epochs": "7", 
            "batch-size": "64", 
            "lr": "0.05"
}

In [22]:
estimator = PyTorch(
    entry_point="train.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.g4dn.xlarge",
    hyperparamenters=hyperparameters,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [26]:
estimator.fit(
    {
        "train":"s3://sagemaker-us-east-1-553171274417/sagemaker/Final-Project/train/",
        "test":"s3://sagemaker-us-east-1-553171274417/sagemaker/Final-Project/test/",
    }
) 

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-06-26-01-39-44-121


Using provided s3_resource
2023-06-26 01:39:44 Starting - Starting the training job......
2023-06-26 01:40:27 Starting - Preparing the instances for training......
2023-06-26 01:41:31 Downloading - Downloading input data...
2023-06-26 01:42:11 Training - Downloading the training image.....................
2023-06-26 01:45:32 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-06-26 01:45:43,731 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-06-26 01:45:43,760 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-06-26 01:45:43,764 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-06-26 01:45:44,030 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs"

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [12]:
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.01, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128]),
    "epochs": IntegerParameter(1, 10),
}

In [13]:
estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.g4dn.xlarge"
)

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [14]:
tuner.fit(
          {
            "train":"s3://sagemaker-us-east-1-553171274417/sagemaker/Final-Project/train/",
            "test":"s3://sagemaker-us-east-1-553171274417/sagemaker/Final-Project/test/",
          }
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating hyperparameter tuning job with name: pytorch-training-230625-2330


Using provided s3_resource
....................................................................................................................................................................................................*


UnexpectedStatusException: Error for HyperParameterTuning job pytorch-training-230625-2330: Failed. Reason: No objective metrics found after running 4 training jobs. Please ensure that the custom algorithm is emitting the objective metric as defined by the regular expression provided.

In [ ]:
best_estimator = tuner.best_estimator()
best_estimator.hyperparameters()

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [ ]:
# TODO: Deploy your model to an endpoint

In [ ]:
# TODO: Run an prediction on the endpoint

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [ ]:
# TODO: Cost Analysis

In [ ]:
# TODO: Train your model using a spot instance

### Multi-Instance Training
**TODO:** Can you train your model on multiple instances?

In [ ]:
# TODO: Train your model on Multiple Instances